In [ ]:
import vectorbt as vbt
import pandas as pd
from datetime import datetime

In [ ]:
def pegar_data_trade(index : int, df : pd.DataFrame) -> datetime:
    return df.index[index]

In [ ]:
def pegar_retorno_acumulado(date : datetime, retorno_acumulado : pd.DataFrame) -> float:
    return retorno_acumulado.loc[date][['retorno_acumulado']]

In [ ]:
ticker = 'PETR4.SA'
data_inicio = '2020-01-15'
data_fim = '2021-12-31'
intervalo = '1d'

In [ ]:
janela_rapida = 15
janela_lenta = 40
patrimonio = 10000

In [ ]:
colunas = ['Close', 'Volume']

In [ ]:
df : pd.DataFrame = vbt.YFData.download(ticker, start=data_inicio, end=data_fim, interval='1h').get(colunas)

In [ ]:
mm_rapida = vbt.MA.run(df['Close'], janela_rapida)
mm_lenta = vbt.MA.run(df['Close'], janela_lenta)

In [ ]:
entradas = mm_rapida.ma_crossed_above(mm_lenta)
saidas = mm_rapida.ma_crossed_below(mm_lenta)

In [ ]:
df['entradas'] = entradas
df['saidas'] = saidas

In [ ]:
portfolio = vbt.Portfolio.from_signals(df['Close'], entradas, saidas, init_cash=patrimonio)

In [ ]:
retorno_acumulado = pd.DataFrame()
retorno_acumulado['retorno_acumulado'] = portfolio.cumulative_returns()

In [ ]:
trades : pd.DataFrame = portfolio.get_trades().records

In [ ]:
trades['trade_entrada'] = trades.apply(lambda x: pegar_data_trade(int(x['entry_idx']), df), axis = 1)

In [ ]:
trades['trade_saida'] = trades.apply(lambda x: pegar_data_trade(int(x['exit_idx']), df), axis = 1)

In [ ]:
trades_view = trades[['trade_entrada', 'trade_saida', 'size', 'exit_price', 'pnl', 'return']]

In [ ]:
trades_view['retorno_acumulado'] = trades_view.apply(lambda x: pegar_retorno_acumulado(x['trade_saida'], retorno_acumulado), axis = 1)

In [ ]:
portfolio.plot_drawdowns().write_image(f'./drawdowns/{ticker}.jpeg')
portfolio.plot_cum_returns().write_image(f'./retorno_acumulado/{ticker}.jpeg')
portfolio.plot_orders().write_image(f'./trades/{ticker}.jpeg')

In [ ]:
data_texto = '%d/%m/%Y %H:%M:S'

In [ ]:
trades_view['trade_entrada'] = trades_view['trade_entrada'].apply(lambda x: x.strftime(data_texto))
trades_view['trade_saida'] = trades_view['trade_saida'].apply(lambda x: x.strftime(data_texto))

In [ ]:
trades_view.to_excel(f'{ticker}.xlsx', index = False)